In [1]:
import numpy as np
np.random.seed(71)

import matplotlib
matplotlib.use('Agg')

from keras import backend as K
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.callbacks import Callback
from keras.utils import np_utils
from keras.objectives import categorical_crossentropy
from keras.datasets import cifar10, mnist

import matplotlib.pyplot as plt
from matplotlib.animation import ArtistAnimation

import multiprocessing as mp
import math

Using TensorFlow backend.


In [2]:
def Hbeta(D, beta):
  P = np.exp(-D * beta)
  sumP = np.sum(P) + 10e-15
  
  H = np.log(sumP) + beta * np.sum(D * P) / sumP
  P = P / sumP
  return H, P

In [3]:
def x2p_job(data):
    i, Di, tol, logU = data
    beta = 1.0
    betamin = -np.inf
    betamax = np.inf
    H, thisP = Hbeta(Di, beta)

    Hdiff = H - logU
    tries = 0
    while np.abs(Hdiff) > tol and tries < 50:
        if Hdiff > 0:
            betamin = beta
            if betamax == -np.inf:
                beta = beta * 2
            else:
                beta = (betamin + betamax) / 2
        else:
            betamax = beta
            if betamin == -np.inf:
                beta = beta / 2
            else:
                beta = (betamin + betamax) / 2

        H, thisP = Hbeta(Di, beta)
        Hdiff = H - logU
        tries += 1

    return i, thisP

In [4]:
def x2p(X):
    tol = 1e-5
    n = X.shape[0]
    logU = np.log(perplexity)

    sum_X = np.sum(np.square(X), axis=1)
    D = sum_X + (sum_X.reshape([-1, 1]) - 2 * np.dot(X, X.T))

    idx = (1 - np.eye(n)).astype(bool)
    D = D[idx].reshape([n, -1])

    def generator():
        for i in range(n):
            yield i, D[i], tol, logU

    pool = mp.Pool(n_jobs)
    result = pool.map(x2p_job, generator())
    P = np.zeros([n, n])
    for i, thisP in result:
        P[i, idx[i]] = thisP

    return P

In [5]:
def calculate_P(X):
    print ("Computing pairwise distances...")
    n = X.shape[0]
    P = np.zeros([n, batch_size])
    for i in range(0, n, batch_size):
        P_batch = x2p(X[i:i + batch_size])
        P_batch[np.isnan(P_batch)] = 0
        P_batch = P_batch + P_batch.T
        P_batch = P_batch / P_batch.sum()
        P_batch = np.maximum(P_batch, 1e-12)
        P[i:i + batch_size] = P_batch
    return P

In [6]:
def KLdivergence(P, Y):
    low_dim = 2
    alpha = low_dim - 1.
    sum_Y = K.sum(K.square(Y), axis=1)
    eps = K.variable(10e-15)
    D = sum_Y + K.reshape(sum_Y, [-1, 1]) - 2 * K.dot(Y, K.transpose(Y))
    Q = K.pow(1 + D / alpha, -(alpha + 1) / 2)
    Q *= K.variable(1 - np.eye(batch_size))
    Q /= K.sum(Q)
    Q = K.maximum(Q, eps)
    C = K.log(P) - K.log(Q)
    C = K.sum(P * C)
    return C

In [7]:
def LoadData():
  print ("Loading cifar-10")
  (X_train, y_train), (X_test, y_test) = cifar10.load_data()

  dataset = (X_train, y_train), (X_test, y_test)
  return dataset

In [8]:
def Reshape(dataset):
  (X_train, y_train), (X_test, y_test) = dataset
  n, channel, row, col = X_train.shape
  X_train = X_train.reshape(-1, channel * row * col)
  X_test = X_test.reshape(-1, channel * row * col)
  
  print ("X_train.shape:", X_train.shape)
  print ("X_test.shape:", X_test.shape)

  dataset = (X_train, y_train), (X_test, y_test), n
  return dataset

In [9]:
def Normalize(dataset):
  (X_train, y_train), (X_test, y_test), n = dataset
  print ("Max in training data before scaling:", str(np.max(X_train)))
  
  X_train = X_train.astype('float32')
  X_test = X_test.astype('float32')
  X_train /= 255
  X_test /= 255
  minimum = np.min(X_train)
  maximum = np.max(y_train)
  
  print ("Max in training data after scaling:", str(np.max(X_train)))
  
  dataset = (X_train, y_train), (X_test, y_test), n
  return dataset

In [10]:
def CompileModel(model, loss = KLdivergence, optimizer = "adam"):
  model.compile(loss=loss, optimizer=optimizer)

In [11]:
def FitModel():
  (X_train, y_train), (X_test, y_test), n = dataset
  
  n_jobs = 4
  batch_size = 1000
  
  low_dim = 2
  nb_epoch = 50
  shuffle_interval = nb_epoch + 1
  
  batch_num = int(n // batch_size)
  m = batch_num * batch_size
  
  images = []
  fig = plt.figure(figsize=(5, 5))

  for epoch in range(nb_epoch):
    # shuffle    
    if epoch % shuffle_interval == 0:
      X = X_train[np.random.permutation(n)[:m]]
      P = calculate_P(X)

    # train
    loss = 0
    for i in range(0, n, batch_size):
        loss += model.train_on_batch(X[i:i+batch_size], P[i:i+batch_size])
    
    print ("Epoch:"+str(epoch+1)+"/"+str(nb_epoch)+" loss: "+str(loss/batch_num))

    # visualize training
    pred = model.predict(X_test)
    img = plt.scatter(pred[:, 0], pred[:, 1], c=y_test, marker='o', s=3, edgecolor='')
    images.append([img])
    
  # plot
  plt.clf()
  fig = plt.figure(figsize=(5, 5))
  pred = model.predict(X_test)
  plt.scatter(pred[:, 0], pred[:, 1], c=y_test, marker='o', s=4, edgecolor='')
  fig.tight_layout()
  plt.savefig("mlp_result.png") 

In [12]:
def GenerateBatch():
    for i in range(0, n, batch_size):
        P = calculate_P(X_train)
        yield(X_train[i:i+batch_size], P[i:i+batch_size])


In [13]:
def AltFit():
  (X_train, y_train), (X_test, y_test), n = dataset
  
  nb_epoch = 50
    
  history = model.fit_generator(GenerateBatch(),steps_per_epoch=math.ceil(n/batch_size), epochs=nb_epoch)
  return history


In [ ]:
dataset = Normalize(Reshape(LoadData()))

Loading cifar-10
123625472/170498071 [====================>.........] - ETA: 35s

In [15]:
(X_train, y_train), (X_test, y_test), n = dataset
X_train = X_train[0:10000]
y_train = y_train[0:10000]

perplexity = 30.0
n_jobs = 4
batch_size = 100

nb_epoch = 50
shuffle_interval = nb_epoch + 1

batch_num = int(n // batch_size)
m = batch_num * batch_size

# build model
model = Sequential()
model.add(Dense(500, input_shape=(X_train.shape[1],)))
model.add(Activation('relu'))

model.add(Dense(500))
model.add(Activation('relu'))

model.add(Dense(1000))
model.add(Activation('relu'))

model.add(Dense(2))
model.compile(loss=KLdivergence, optimizer="adam")
  

W0811 22:40:31.442429 139889707059072 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0811 22:40:32.634369 139889707059072 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0811 22:40:32.646214 139889707059072 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0811 22:40:32.719790 139889707059072 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0811 22:40:32.776007 139889707059072 deprecation_wrappe

In [0]:
# Plot the training and validation loss + accuracy
def plot_training(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    epochs = range(len(acc))

    plt.plot(epochs, acc, 'r.')
    plt.plot(epochs, val_acc, 'r')
    plt.title('Training and validation accuracy')

    plt.show()

    plt.savefig('acc_vs_epochs.png')

# In[9]:


In [0]:
history = AltFit()


W0811 22:40:32.898920 139889707059072 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0811 22:40:33.282837 139889707059072 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/50
Computing pairwise distances...
Computing pairwise distances...
  2/500 [..............................] - ETA: 3:25:27 - loss: 0.8144Computing pairwise distances...


In [0]:

plot_training(history)